In [18]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from Dataset.WindowDataset import WindowDataset
from Model.AE import Auto
from Model.LSTM import LSTM
from utils import combine_tensors

In [24]:
eco = '콜금리'
batch_size = 4
max_apart_size = 70  # 0~60평 아파트만 돌림
real_estate_weighted_average = 0.5
input_size = 5 # LSTM input_size
hidden_size = 16 # LSTM hidden_size

In [8]:
def make_estimate(x):
    vec = np.zeros((max_apart_size))
    vec[int(x[0])] = x[1]
    return vec

In [9]:
df = pd.read_csv('../데이터/merge.csv')
df['시점'] = df['계약년월'] + df['계약일']
df = df[:100] # //////////////////////////////
df['시점'].apply(lambda x : int(x))
df['평수'] = df['전용면적(㎡)'].apply(lambda x : x // 3.3058)
df['가격'] = df['거래금액(만원)'].apply(lambda x : int(x.replace(',','')))
df = df.sort_values(by=['시점'])
df['부동산'] = df[['평수','가격']].apply(make_estimate,axis=1)
df.reset_index(drop=True)
df['경제'] = df[eco]
df = df[['시점','부동산','경제']]

In [12]:
len_df = len(df)
train_len = int(len_df *0.6)
val_len = int(len_df * 0.3)

batch_size = 2

train_dataset = WindowDataset(df[:train_len])
train_loader = DataLoader(train_dataset, batch_size=batch_size)

val_dataset = WindowDataset(df[train_len:train_len+val_len])
val_loader = DataLoader(val_dataset, batch_size=batch_size)

test_dataset = WindowDataset(df[train_len+val_len:])
test_loader = DataLoader(test_dataset, batch_size=batch_size)

c:\Users\hkyoo\OneDrive\바탕 화면\SCI\코드\Dataset\WindowDataset.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  self.x1 = torch.FloatTensor(X1)


In [16]:
for x1,x2,y1,y2 in train_loader:
    print(x1.shape)
    print(x2.shape)
    break

torch.Size([2, 5, 2])
torch.Size([2, 5, 1, 70])


In [28]:
######### 모델


In [27]:
#############################
# 데이터 & 모델에 device 붙임!!!
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

# model = Model.to(device)

# print('작동하는지 실험')
# basic_data = torch.rand((1,input_size))
# model(basic_data)

criterion = nn.MSELoss()

lr = 1e-3
num_epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

cpu is available


NameError: name 'model' is not defined

In [ ]:
economy_embedding_model = LSTM(input_size,hidden_size).to(device)
economy_embedding_model.load_state_dict(torch.load('Checkpoint/economic_best_model.pth'))

real_estate_embedding_model = Auto(max_apart_size).to(device)
real_estate_embedding_model.load_state_dict(torch.load('Checkpoint/embedding_best_model.pth'))

In [ ]:
for epoch in range(num_epochs):
    print('epoch : {}'.format(epoch))
    running_loss = 0.0
    num = 0
    print("train")
    model.train()
    for x1, x2, y1, y2 in train_loader:
        x1, x2 = x1.to(device), x2.to(device) # x1: [시점, 경제], x2: 부동산
        y1, y2 = y1.to(device), y2.to(device)

        # 경제 데이터
        eco_emb = economy_embedding_model(x1[1])
        # 부동산 데이터
        real_est_emb = real_estate_embedding_model(x2)
        
        final_emb = combine_tensors(real_est_emb, eco_emb, real_estate_weighted_average, 'sum')
        ##########################
        # 모델에 x[0], final_emb 가지고 ODE 학습하기!!!!